In [ ]:
# Google Colabでライブラリをアップロードする
from google.colab import files
uploaded = files.upload()

In [ ]:
# Google Colabでドライブのデータを使う
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Google Colabでライブラリをインストールする
!pip install japanize_matplotlib bottleneck tslearn

In [ ]:
# 自作関数
import MasterResearchFunction as mr

# 基本ライブラリ
import os, re, csv, math, statistics
from datetime import datetime, timedelta
from decimal import Decimal
import pickle

# 数値計算とデータ処理
import numpy as np
import pandas as pd
import bottleneck as bn

# 機械学習ライブラリ
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# ディープラーニングライブラリ
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation, Conv1D, MaxPooling1D, Flatten

# プロットと可視化
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import japanize_matplotlib

# その他のライブラリ
from scipy import signal, stats
from scipy.signal import savgol_filter
from scipy.spatial.distance import euclidean
from scipy.interpolate import interp1d
from tslearn.metrics import dtw_path
from fastdtw import fastdtw

In [ ]:
#Apple Watchのモーションデータの読み込み
a_yuuma_motion_data = mr.process_apple_watch_csv("datasets/yuuma/20240604/yuuma_motion.csv")
b_sakamoto_motion_data = mr.process_apple_watch_csv("datasets/sakamoto/20240604/sakamoto_motion.csv")
c_watabe_motion_data = mr.process_apple_watch_csv("datasets/watabe/20240605/watabe_motion.csv")
d_nakazawa_motion_data = mr.process_apple_watch_csv("datasets/nakazawa/20240606/nakazawa_motion.csv")
e_okede_motion_data = mr.process_apple_watch_csv("datasets/okeda/20240607/okeda_motion.csv")
# a_yuuma_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/yuuma/20240604/yuuma_motion.csv")
# b_sakamoto_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/sakamoto/20240604/sakamoto_motion.csv")
# c_watabe_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/watabe/20240605/watabe_motion.csv")
# d_nakazawa_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/nakazawa/20240606/nakazawa_motion.csv")
# e_okeda_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/okeda/20240607/okeda_motion.csv")

In [ ]:
#Tobiiのモーションデータの読み込み
a_yuuma_eye_data = mr.process_tobii_csv("datasets/yuuma/20240604/yuuma_eye.csv")
b_sakamoto_eye_data = mr.process_tobii_csv("datasets/sakamoto/20240604/sakamoto_eye.csv")
c_watabe_eye_data = mr.process_tobii_csv("datasets/watabe/20240605/watabe_eye.csv")
d_nakazawa_eye_data = mr.process_tobii_csv("datasets/nakazawa/20240606/nakazawa_eye.csv")
e_okeda_eye_data = mr.process_tobii_csv("datasets/okeda/20240607/okeda_eye.csv")
# a_yuuma_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/yuuma/20240604/yuuma_eye.csv")
# b_sakamoto_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/sakamoto/20240604/sakamoto_eye.csv")
# c_watabe_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/watabe/20240605/watabe_eye.csv")
# d_nakazawa_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/nakazawa/20240606/nakazawa_eye.csv")
# e_okeda_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/okeda/20240607/okeda_eye.csv")

In [ ]:
#a_yuumaくんの教師データ読み込み
a_yuuma_check = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/check")
a_yuuma_circle = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/circle")
a_yuuma_cross = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/cross(new)")
a_yuuma_tri = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/tri")
# a_yuuma_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/check")
# a_yuuma_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/circle")
# a_yuuma_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/cross(new)")
# a_yuuma_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/tri")

In [ ]:
#b_sakamotoくんの教師データ読み込み
b_sakamoto_check = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(old)/check")
b_sakamoto_circle = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(old)/circle")
b_sakamoto_cross = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(old)/cross")
b_sakamoto_tri = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(old)/tri")
# b_sakamoto_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(old)/check")
# b_sakamoto_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(old)/circle")
# b_sakamoto_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(old)/cross")
# b_sakamoto_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(old)/tri")

In [ ]:
#b_sakamotoくんの教師データ読み込み
b_sakamoto_check = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(new)/check")
b_sakamoto_circle = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(new)/circle")
b_sakamoto_cross = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(new)/cross")
b_sakamoto_tri = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture(new)/tri")
# b_sakamoto_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(new)/check")
# b_sakamoto_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(new)/circle")
# b_sakamoto_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(new)/cross")
# b_sakamoto_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture(new)/tri")

In [ ]:
#c_watabeくんの教師データ読み込み
c_watabe_check = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/check")
c_watabe_circle = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/circle")
c_watabe_cross = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/cross")
c_watabe_tri = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/tri")
# c_watabe_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/check")
# c_watabe_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/circle")
# c_watabe_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/cross")
# c_watabe_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/tri")

In [ ]:
#d_nakazawaくんの教師データ読み込み
d_nakazawa_check = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/check")
d_nakazawa_circle = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/circle")
d_nakazawa_cross = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/cross")
d_nakazawa_tri = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/tri")
# d_nakazawa_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/check")
# d_nakazawa_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/circle")
# d_nakazawa_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/cross")
# d_nakazawa_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/tri")

In [ ]:
#e_okedaくんの教師データ読み込み
e_okeda_check = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/check")
e_okeda_circle = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/circle")
e_okeda_cross = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/cross")
e_okeda_tri = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/tri")
# e_okeda_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/check")
# e_okeda_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/circle")
# e_okeda_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/cross")
# e_okeda_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/tri")

In [ ]:
circle_start_time = pd.Timestamp('2024-06-04 16:00:31.500')
circle_end_time = pd.Timestamp('2024-06-04 16:00:34.000')
cross_start_time = pd.Timestamp('2024-06-04 16:10:56.000')
cross_end_time = pd.Timestamp('2024-06-04 16:10:58.000')
tri_start_time = pd.Timestamp('2024-06-04 16:31:37.000')
tri_end_time = pd.Timestamp('2024-06-04 16:31:39.750')
check_start_time = pd.Timestamp('2024-06-04 16:46:43.500')
check_end_time = pd.Timestamp('2024-06-04 16:46:45.000')


In [ ]:
data = a_yuuma_motion_data[
    (a_yuuma_motion_data['Timestamp'] >= (check_start_time - pd.Timedelta(seconds=5))) &
    (a_yuuma_motion_data['Timestamp'] <= (check_end_time + pd.Timedelta(seconds=5)))
]

In [ ]:
plt.plot(data['GyroX'])
plt.plot(data['GyroY'])
plt.plot(data['GyroZ'])
plt.ylim([-5, 5]) #X軸調整
print(len(data))
plt.show

for i in range(len(c_watabe_cross)):
  plt.figure()
  plt.plot(c_watabe_cross[i]['GyroX'])
  plt.plot(c_watabe_cross[i]['GyroY'])
  plt.plot(c_watabe_cross[i]['GyroZ'])
  plt.ylim([-5, 5]) #X軸調整

  print(len(c_watabe_cross[i]))
  plt.show

In [ ]:
a = []
for i in range(5):
  print(len(a_yuuma_check[i]))
  a.append(len(a_yuuma_check[i]))
sum(a)/5

In [ ]:
circle_start_time = pd.Timestamp('2024-06-06 13:44:14.000')
circle_end_time = pd.Timestamp('2024-06-06 13:44:16.500')
cross_start_time = pd.Timestamp('2024-06-06 13:54:44.000')
cross_end_time = pd.Timestamp('2024-06-06 13:54:46.500')
tri_start_time = pd.Timestamp('2024-06-06 14:14:52.000')
tri_end_time = pd.Timestamp('2024-06-06 14:14:54.500')
check_start_time = pd.Timestamp('2024-06-06 14:30:23.250')
check_end_time = pd.Timestamp('2024-06-06 14:30:24.500')

data = d_nakazawa_motion_data[
    (d_nakazawa_motion_data['Timestamp'] >= (check_start_time - pd.Timedelta(seconds=5))) &
    (d_nakazawa_motion_data['Timestamp'] <= (check_end_time + pd.Timedelta(seconds=5)))
]

In [ ]:
train_data = d_nakazawa_motion_data[
    (d_nakazawa_motion_data['Timestamp'] >= (check_start_time - pd.Timedelta(seconds=0.2))) &
    (d_nakazawa_motion_data['Timestamp'] <= (check_end_time + pd.Timedelta(seconds=0.2)))
]

In [ ]:
# データをスケーリングする関数
def scale_data(data, scale_factor):
    return data * scale_factor

def add_noise(data, noise_level=0.1):
    noise = np.random.normal(0, noise_level, size=data.shape)
    return data + noise


train_data['AccelerationX'] = scale_data(train_data['AccelerationX'], 1.08)
train_data['AccelerationX'] = add_noise(train_data['AccelerationX'], noise_level=0.055)
train_data['GyroX'] = scale_data(train_data['GyroX'], 1.08)
train_data['GyroX'] = add_noise(train_data['GyroX'], noise_level=0.055)

train_data['AccelerationY'] = scale_data(train_data['AccelerationY'], 1.11)
train_data['AccelerationY'] = add_noise(train_data['AccelerationY'], noise_level=0.058)
train_data['GyroY'] = scale_data(train_data['GyroY'], 1.11)
train_data['GyroY'] = add_noise(train_data['GyroY'], noise_level=0.058)

train_data['AccelerationZ'] = scale_data(train_data['AccelerationZ'], 0.99)
train_data['AccelerationZ'] = add_noise(train_data['AccelerationZ'], noise_level=0.063)
train_data['GyroZ'] = scale_data(train_data['GyroZ'], 0.99)
train_data['GyroZ'] = add_noise(train_data['GyroZ'], noise_level=0.063)

In [ ]:
len(train_data)

In [ ]:
d_nakazawa_check[2]['AccelerationX'] = train_data['AccelerationX'].reset_index(drop = True)
d_nakazawa_check[2]['AccelerationY'] = train_data['AccelerationY'].reset_index(drop = True)
d_nakazawa_check[2]['AccelerationZ'] = train_data['AccelerationZ'].reset_index(drop = True)
d_nakazawa_check[2]['GyroX'] = train_data['GyroX'].reset_index(drop = True)
d_nakazawa_check[2]['GyroY'] = train_data['GyroY'].reset_index(drop = True)
d_nakazawa_check[2]['GyroZ'] = train_data['GyroZ'].reset_index(drop = True)

In [ ]:
d_nakazawa_check[2]

In [ ]:
selected_columns = d_nakazawa_check[2][['UnixTime', 'AccelerationX', 'AccelerationY', 'AccelerationZ', 'GyroX', 'GyroY', 'GyroZ']]  # 'Name'と'Age'カラムのみを選択

In [ ]:
selected_columns.to_csv('datasets/nakazawa/train_gesture/check/MotionData_20240811_142113.csv', index=False)

In [ ]:
Th = 2.6
Th1 = 10
Th2 = 10
Th3 = 10
min_time = 1
max_time = 5

In [ ]:
segx = mr.spring_ogawa(data['AccelerationX'].reset_index(drop = True), train_data['AccelerationX'].reset_index(drop = True), 10)
segy = mr.spring_ogawa(data['AccelerationY'].reset_index(drop = True), train_data['AccelerationY'].reset_index(drop = True), 10)
segz = mr.spring_ogawa(data['AccelerationZ'].reset_index(drop = True), train_data['AccelerationZ'].reset_index(drop = True), 10)

In [ ]:
segx

In [ ]:
segy

In [ ]:
segz

In [ ]:
data['Timestamp'].reset_index(drop=True)[262]
#255~422

In [ ]:
plt.plot(data['AccelerationX'])
plt.plot(data['AccelerationY'])
plt.plot(data['AccelerationZ'])
plt.ylim([-1, 1]) #X軸調整
plt.show

In [ ]:
plt.plot(train_data['AccelerationX'])
plt.plot(train_data['AccelerationY'])
plt.plot(train_data['AccelerationZ'])
plt.ylim([-1, 1]) #X軸調整
plt.show

In [ ]:
len(data)

In [ ]:

for i in range(len(a_yuuma_cross)):
  plt.figure()
  plt.plot(a_yuuma_cross[i]['AccelerationX'])
  plt.plot(a_yuuma_cross[i]['AccelerationY'])
  plt.plot(a_yuuma_cross[i]['AccelerationZ'])
  plt.ylim([-1, 1]) #X軸調整

  print(len(a_yuuma_cross[i]))
  plt.show

In [ ]:
for i in range(len(a_yuuma_cross)):
  plt.figure()
  plt.plot(a_yuuma_cross[i]['AccelerationX'])
  plt.plot(a_yuuma_cross[i]['AccelerationY'])
  plt.plot(a_yuuma_cross[i]['AccelerationZ'])
  plt.ylim([-1, 1]) #X軸調整

  print(len(a_yuuma_cross[i]))
  plt.show

In [ ]:
circle_start_time = pd.Timestamp('2024-06-04 16:00:31.500')
circle_end_time = pd.Timestamp('2024-06-04 16:00:34.000')
cross_start_time = pd.Timestamp('2024-06-04 16:10:56.000')
cross_end_time = pd.Timestamp('2024-06-04 16:10:58.000')
tri_start_time = pd.Timestamp('2024-06-04 16:31:37.000')
tri_end_time = pd.Timestamp('2024-06-04 16:31:39.750')
check_start_time = pd.Timestamp('2024-06-04 16:46:43.500')
check_end_time = pd.Timestamp('2024-06-04 16:46:45.000')

data = a_yuuma_motion_data[
    (a_yuuma_motion_data['Timestamp'] >= (check_start_time - pd.Timedelta(seconds=5))) &
    (a_yuuma_motion_data['Timestamp'] <= (check_end_time + pd.Timedelta(seconds=5)))
]

In [ ]:
Th = 2.6
Th1 = 3.5
Th2 = 3.5
Th3 = 3.5
min_time = 1
max_time = 5

In [ ]:
segx, segy, segz = [], [], []
for i in range(len(a_yuuma_tri)):
    segx.append(mr.spring_ogawa(data['AccelerationX'].reset_index(drop = True), a_yuuma_tri[i]['AccelerationX'], Th1))
    segy.append(mr.spring_ogawa(data['AccelerationY'].reset_index(drop = True), a_yuuma_tri[i]['AccelerationY'], Th2))
    segz.append(mr.spring_ogawa(data['AccelerationZ'].reset_index(drop = True), a_yuuma_tri[i]['AccelerationZ'], Th3))

In [ ]:
a_yuuma_cross_1 = mr.process_apple_watch_csv('datasets/yuuma/train_gesture/cross(new)/MotionData_20240721_160867.csv')

In [ ]:
segx, segy, segz = mr.three_axis_spring_ogawa(a_yuuma_motion_data, a_yuuma_cross, 4.5, 4.5, 4.5, 'acc')

In [ ]:
segx

In [ ]:
segy

In [ ]:
segz

In [ ]:
data['Timestamp'].reset_index(drop=True)[386]
#255~422

In [ ]:
min_time = 1
max_time = 4

In [ ]:
segx = mr.filter_seg_by_elapsed_time(segx, a_yuuma_motion_data, min_time, max_time)
segy = mr.filter_seg_by_elapsed_time(segy, a_yuuma_motion_data, min_time, max_time)
segz = mr.filter_seg_by_elapsed_time(segz, a_yuuma_motion_data, min_time, max_time)

In [ ]:
segx

In [ ]:
segy

In [ ]:
segz

In [ ]:
# 各教師データの結果ごとにオーバーラップを検出
overlap = []
for i in range(len(segx)):
    overlap.append(mr.combine_and_find_overlapping_filtered_segments(segx[i], segy[i], segz[i]))

In [ ]:
overlap

In [ ]:
filtered_overlap = mr.filter_overlap_by_elapsed_time(overlap, a_yuuma_motion_data, min_time, max_time)

In [ ]:
filtered_overlap

In [ ]:
combine_overlap = mr.new_combine_all_overlaps(filtered_overlap)

In [ ]:
combine_overlap

In [ ]:
results =  mr.extract_timestamp_from_overlap(a_yuuma_motion_data, combine_overlap)

In [ ]:
results

In [ ]:
min_time

In [ ]:
max_time

In [ ]:
Th1, Th2, Th3

In [ ]:
#  pickleファイルを読み込む
with open('/Users/hinase/Downloads/Th-s/d_nakazawa_acc_check_segments4.7new.pkl', 'rb') as f:
    data = pickle.load(f)

segx = data['d_nakazawa_check_segx']
segy = data['d_nakazawa_check_segy']
segz = data['d_nakazawa_check_segz']

In [ ]:
min_time = 0.5
max_time = 2.5

In [ ]:
filtered_segx = mr.filter_seg_by_elapsed_time(segx, d_nakazawa_motion_data, min_time, max_time)
filtered_segy = mr.filter_seg_by_elapsed_time(segy, d_nakazawa_motion_data, min_time, max_time)
filtered_segz = mr.filter_seg_by_elapsed_time(segz, d_nakazawa_motion_data, min_time, max_time)

In [ ]:
filtered_segx

In [ ]:
filtered_segy

In [ ]:
filtered_segz

In [ ]:
#フィルタリングされたセグメントを統合
def a_combine_and_find_overlapping_filtered_segments(segx, segy, segz):
    # Combine all segments
    all_segments = []
    for seg in [segx, segy, segz]:
        for (t_s, t_e) in seg:
            all_segments.append((t_s, t_e))

    # Sort segments
    all_segments.sort()

    # Find overlapping segments
    overlap_ranges = []
    current_overlap = None
    current_count = 0

    for start, end in all_segments:
        if current_overlap is None:
            current_overlap = (start, end)
            current_count = 1
        else:
            current_start, current_end = current_overlap
            if start <= current_end:
                current_overlap = (current_start, max(current_end, end))
                current_count += 1
            else:
                if current_count >= 2:
                    overlap_ranges.append(current_overlap)
                current_overlap = (start, end)
                current_count = 1

    if current_count >= 2:
        overlap_ranges.append(current_overlap)

    return overlap_ranges

In [ ]:
# 各教師データの結果ごとにオーバーラップを検出
overlap = []
for i in range(len(filtered_segx)):
    overlap.append(a_combine_and_find_overlapping_filtered_segments(filtered_segx[i], filtered_segy[i], filtered_segz[i]))

In [ ]:
overlap

In [ ]:
filtered_overlap = mr.filter_overlap_by_elapsed_time(overlap, d_nakazawa_motion_data, min_time, max_time)

In [ ]:
filtered_overlap

In [ ]:
d_nakazawa_combine_overlap = mr.new_combine_all_overlaps(filtered_overlap)

In [ ]:
d_nakazawa_combine_overlap

In [ ]:
d_nakazawa_results =  mr.extract_timestamp_from_overlap(d_nakazawa_motion_data, d_nakazawa_combine_overlap)

In [ ]:
d_nakazawa_results

In [ ]:
d_nakazawa_check_filtered_results = mr.filter_segments_by_time_length(d_nakazawa_results, 1.5, 1.75)

In [ ]:
d_nakazawa_check_filtered_results

In [ ]:
len(d_nakazawa_check_filtered_results)

In [ ]:
a_yuuma_circle_filtered_results
a_yuuma_cross_filtered_results
a_yuuma_check_filtered_results
b_sakamoto_circle_filtered_results
b_sakamoto_check_filtered_results
c_watabe_circle_filtered_results
c_watabe_cross_filtered_results
c_watabe_tri_filtered_results
c_watabe_check_filtered_results
d_nakazawa_circle_filtered_results
d_nakazawa_check_filtered_results

In [ ]:
import pickle

# 保存する変数を辞書にまとめる
data = {
    'a_yuuma_circle_filtered_results': a_yuuma_circle_filtered_results,
    'a_yuuma_cross_filtered_results': a_yuuma_cross_filtered_results,
    #'a_yuuma_tri_filtered_results': a_yuuma_tri_filtered_results,
    'a_yuuma_check_filtered_results': a_yuuma_check_filtered_results,
    'b_sakamoto_circle_filtered_results': b_sakamoto_circle_filtered_results,
    #'b_sakamoto_cross_filtered_results': b_sakamoto_cross_filtered_results,
    #'b_sakamoto_tri_filtered_results': b_sakamoto_tri_filtered_results,
    'b_sakamoto_check_filtered_results': b_sakamoto_check_filtered_results,
    'c_watabe_circle_filtered_results': c_watabe_circle_filtered_results,
    'c_watabe_cross_filtered_results': c_watabe_cross_filtered_results,
    'c_watabe_tri_filtered_results': c_watabe_tri_filtered_results,
    'c_watabe_check_filtered_results': c_watabe_check_filtered_results,
    'd_nakazawa_circle_filtered_results': d_nakazawa_circle_filtered_results,
    #'d_nakazawa_cross_filtered_results': d_nakazawa_cross_filtered_results,
    #'d_nakazawa_tri_filtered_results': d_nakazawa_tri_filtered_results,
    'd_nakazawa_check_filtered_results': d_nakazawa_check_filtered_results,

}

# 変数をpickleファイルに保存
with open('filter_results_segments.pkl', 'wb') as f:
    pickle.dump(data, f)

In [ ]:
#  pickleファイルを読み込む
with open('filter_results_segments.pkl', 'rb') as f:
    data = pickle.load(f)

a_yuuma_circle_filtered_results = data['a_yuuma_circle_filtered_results']
a_yuuma_cross_filtered_results = data['a_yuuma_cross_filtered_results']
a_yuuma_check_filtered_results = data['a_yuuma_check_filtered_results']
b_sakamoto_circle_filtered_results = data['b_sakamoto_circle_filtered_results']
b_sakamoto_check_filtered_results = data['b_sakamoto_check_filtered_results']
c_watabe_circle_filtered_results = data['c_watabe_circle_filtered_results']
c_watabe_cross_filtered_results = data['c_watabe_cross_filtered_results']
c_watabe_tri_filtered_results = data['c_watabe_tri_filtered_results']
c_watabe_check_filtered_results = data['c_watabe_check_filtered_results']
d_nakazawa_circle_filtered_results = data['d_nakazawa_circle_filtered_results']
d_nakazawa_check_filtered_results = data['d_nakazawa_check_filtered_results']

In [ ]:
a_yuuma_cross_filtered_results

In [ ]:
a_yuuma_extracted_eye_data = mr.extract_eye_data_within_intervals(a_yuuma_cross_filtered_results, a_yuuma_eye_data)

In [ ]:
std_devs = []
data = a_yuuma_extracted_eye_data
for i in range(len(data)):
    #視線2D
    diff_x = data[i]['Gaze point X'].interpolate(method='linear').diff()
    diff_y = data[i]['Gaze point Y'].interpolate(method='linear').diff()
    distance_gaze2d = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離

    #固視点
    diff_x = data[i]['Fixation point X'].interpolate(method='linear').diff()
    diff_y = data[i]['Fixation point Y'].interpolate(method='linear').diff()
    distance_fixa = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離

    #視線3D
    diff_x = data[i]['Gaze point 3D X'].interpolate(method='linear').diff()
    diff_y = data[i]['Gaze point 3D Y'].interpolate(method='linear').diff()
    diff_z = data[i]['Gaze point 3D Z'].interpolate(method='linear').diff()
    distance_gaze3d = np.sqrt(diff_x**2 + diff_y**2 + diff_z**2)  # ユークリッド距離

    #瞳孔径変化
    diff_r = data[i]['Pupil diameter right'].interpolate(method='linear').diff()
    diff_l = data[i]['Pupil diameter left'].interpolate(method='linear').diff()
    diff_pupil = (diff_r + diff_l) / 2

    #視線方向
    rx = data[i]['Gaze direction right X'].interpolate(method='linear').diff()
    ry = data[i]['Gaze direction right Y'].interpolate(method='linear').diff()
    rz = data[i]['Gaze direction right Z'].interpolate(method='linear').diff()
    distance_r = np.sqrt(rx**2 + ry**2 + rz**2)
    lx = data[i]['Gaze direction left X'].interpolate(method='linear').diff()
    ly = data[i]['Gaze direction left Y'].interpolate(method='linear').diff()
    lz = data[i]['Gaze direction left Z'].interpolate(method='linear').diff()
    distance_l = np.sqrt(lx**2 + ly**2 + lz**2)
    distance_dire = (distance_l + distance_r) / 2

    #瞳孔位置
    rx = data[i]['Pupil position right X'].interpolate(method='linear').diff()
    ry = data[i]['Pupil position right Y'].interpolate(method='linear').diff()
    rz = data[i]['Pupil position right Z'].interpolate(method='linear').diff()
    distance_r = np.sqrt(rx**2 + ry**2 + rz**2)
    lx = data[i]['Pupil position left X'].interpolate(method='linear').diff()
    ly = data[i]['Pupil position left Y'].interpolate(method='linear').diff()
    lz = data[i]['Pupil position left Z'].interpolate(method='linear').diff()
    distance_l = np.sqrt(lx**2 + ly**2 + lz**2)
    distance_pupilposi = (distance_r + distance_l) / 2

    std_devs.append((distance_gaze2d.std(), distance_fixa.std(), distance_gaze3d.std(), diff_pupil.std(), distance_dire.std(), distance_pupilposi.std()))

# 結果を出力
std_dev_df = pd.DataFrame(std_devs, columns=['Gaze2D', 'Fixation', 'Gaze3D', 'Pupil', 'GazeDire', 'PupilPosi'])
print(std_dev_df)

In [ ]:
std = pd.concat([std_individual_a, std_individual_b, std_individual_c, std_individual_d])

In [ ]:
mean = pd.concat([mean_individual_a, mean_individual_b, mean_individual_c, mean_individual_d])

In [ ]:
1.5 * std.drop('Subjects', axis = 1)

In [ ]:
mean.drop('Subjects', axis = 1) + std.drop('Subjects', axis = 1)

In [ ]:
mean.drop('Subjects', axis = 1) - 0.5 * std.drop('Subjects', axis = 1)

In [ ]:
std + mean

In [ ]:
std_individual_a = []
mean_individual_a = []
#視線2D
diff_x = a_yuuma_eye_data['Gaze point X'].interpolate(method='linear').diff()
diff_y = a_yuuma_eye_data['Gaze point Y'].interpolate(method='linear').diff()
distance_gaze2d = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離

#固視点
diff_x = a_yuuma_eye_data['Fixation point X'].interpolate(method='linear').diff()
diff_y = a_yuuma_eye_data['Fixation point Y'].interpolate(method='linear').diff()
distance_fixa = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離

#視線3D
diff_x = a_yuuma_eye_data['Gaze point 3D X'].interpolate(method='linear').diff()
diff_y = a_yuuma_eye_data['Gaze point 3D Y'].interpolate(method='linear').diff()
diff_z = a_yuuma_eye_data['Gaze point 3D Z'].interpolate(method='linear').diff()
distance_gaze3d = np.sqrt(diff_x**2 + diff_y**2 + diff_z**2)  # ユークリッド距離

#瞳孔径変化
diff_r = a_yuuma_eye_data['Pupil diameter right'].interpolate(method='linear').diff()
diff_l = a_yuuma_eye_data['Pupil diameter left'].interpolate(method='linear').diff()
diff_pupil = (diff_r + diff_l) / 2

#視線方向
rx = a_yuuma_eye_data['Gaze direction right X'].interpolate(method='linear').diff()
ry = a_yuuma_eye_data['Gaze direction right Y'].interpolate(method='linear').diff()
rz = a_yuuma_eye_data['Gaze direction right Z'].interpolate(method='linear').diff()
distance_r = np.sqrt(rx**2 + ry**2 + rz**2)
lx = a_yuuma_eye_data['Gaze direction left X'].interpolate(method='linear').diff()
ly = a_yuuma_eye_data['Gaze direction left Y'].interpolate(method='linear').diff()
lz = a_yuuma_eye_data['Gaze direction left Z'].interpolate(method='linear').diff()
distance_l = np.sqrt(lx**2 + ly**2 + lz**2)
distance_dire = (distance_l + distance_r) / 2

#瞳孔位置
rx = a_yuuma_eye_data['Pupil position right X'].interpolate(method='linear').diff()
ry = a_yuuma_eye_data['Pupil position right Y'].interpolate(method='linear').diff()
rz = a_yuuma_eye_data['Pupil position right Z'].interpolate(method='linear').diff()
distance_r = np.sqrt(rx**2 + ry**2 + rz**2)
lx = a_yuuma_eye_data['Pupil position left X'].interpolate(method='linear').diff()
ly = a_yuuma_eye_data['Pupil position left Y'].interpolate(method='linear').diff()
lz = a_yuuma_eye_data['Pupil position left Z'].interpolate(method='linear').diff()
distance_l = np.sqrt(lx**2 + ly**2 + lz**2)
distance_pupilposi = (distance_r + distance_l) / 2

std_individual_a.append(('a_yuuma', distance_gaze2d.std(), distance_fixa.std(), distance_gaze3d.std(), diff_pupil.std(), distance_dire.std(), distance_pupilposi.std()))
mean_individual_a.append(('a_yuuma', distance_gaze2d.mean(), distance_fixa.mean(), distance_gaze3d.mean(), diff_pupil.mean(), distance_dire.mean(), distance_pupilposi.mean()))

# 結果を出力
std_individual_a = pd.DataFrame(std_individual_a, columns=['Subjects', 'Gaze2D', 'Fixation', 'Gaze3D', 'Pupil', 'GazeDire', 'PupilPosi'])
# 結果を出力
mean_individual_a = pd.DataFrame(mean_individual_a, columns=['Subjects', 'Gaze2D', 'Fixation', 'Gaze3D', 'Pupil', 'GazeDire', 'PupilPosi'])

In [ ]:
std_individual_b = []
mean_individual_b = []
#視線2D
diff_x = b_sakamoto_eye_data['Gaze point X'].interpolate(method='linear').diff()
diff_y = b_sakamoto_eye_data['Gaze point Y'].interpolate(method='linear').diff()
distance_gaze2d = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離

#固視点
diff_x = b_sakamoto_eye_data['Fixation point X'].interpolate(method='linear').diff()
diff_y = b_sakamoto_eye_data['Fixation point Y'].interpolate(method='linear').diff()
distance_fixa = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離

#視線3D
diff_x = b_sakamoto_eye_data['Gaze point 3D X'].interpolate(method='linear').diff()
diff_y = b_sakamoto_eye_data['Gaze point 3D Y'].interpolate(method='linear').diff()
diff_z = b_sakamoto_eye_data['Gaze point 3D Z'].interpolate(method='linear').diff()
distance_gaze3d = np.sqrt(diff_x**2 + diff_y**2 + diff_z**2)  # ユークリッド距離

#瞳孔径変化
diff_r = b_sakamoto_eye_data['Pupil diameter right'].interpolate(method='linear').diff()
diff_l = b_sakamoto_eye_data['Pupil diameter left'].interpolate(method='linear').diff()
diff_pupil = (diff_r + diff_l) / 2

#視線方向
rx = b_sakamoto_eye_data['Gaze direction right X'].interpolate(method='linear').diff()
ry = b_sakamoto_eye_data['Gaze direction right Y'].interpolate(method='linear').diff()
rz = b_sakamoto_eye_data['Gaze direction right Z'].interpolate(method='linear').diff()
distance_r = np.sqrt(rx**2 + ry**2 + rz**2)
lx = b_sakamoto_eye_data['Gaze direction left X'].interpolate(method='linear').diff()
ly = b_sakamoto_eye_data['Gaze direction left Y'].interpolate(method='linear').diff()
lz = b_sakamoto_eye_data['Gaze direction left Z'].interpolate(method='linear').diff()
distance_l = np.sqrt(lx**2 + ly**2 + lz**2)
distance_dire = (distance_l + distance_r) / 2

#瞳孔位置
rx = b_sakamoto_eye_data['Pupil position right X'].interpolate(method='linear').diff()
ry = b_sakamoto_eye_data['Pupil position right Y'].interpolate(method='linear').diff()
rz = b_sakamoto_eye_data['Pupil position right Z'].interpolate(method='linear').diff()
distance_r = np.sqrt(rx**2 + ry**2 + rz**2)
lx = b_sakamoto_eye_data['Pupil position left X'].interpolate(method='linear').diff()
ly = b_sakamoto_eye_data['Pupil position left Y'].interpolate(method='linear').diff()
lz = b_sakamoto_eye_data['Pupil position left Z'].interpolate(method='linear').diff()
distance_l = np.sqrt(lx**2 + ly**2 + lz**2)
distance_pupilposi = (distance_r + distance_l) / 2

std_individual_b.append(('b_sakamoto', distance_gaze2d.std(), distance_fixa.std(), distance_gaze3d.std(), diff_pupil.std(), distance_dire.std(), distance_pupilposi.std()))
mean_individual_b.append(('b_sakamoto', distance_gaze2d.mean(), distance_fixa.mean(), distance_gaze3d.mean(), diff_pupil.mean(), distance_dire.mean(), distance_pupilposi.mean()))

# 結果を出力
std_individual_b = pd.DataFrame(std_individual_b, columns=['Subjects', 'Gaze2D', 'Fixation', 'Gaze3D', 'Pupil', 'GazeDire', 'PupilPosi'])
# 結果を出力
mean_individual_b = pd.DataFrame(mean_individual_b, columns=['Subjects', 'Gaze2D', 'Fixation', 'Gaze3D', 'Pupil', 'GazeDire', 'PupilPosi'])

In [ ]:
std_individual_c = []
mean_individual_c = []
#視線2D
diff_x = c_watabe_eye_data['Gaze point X'].interpolate(method='linear').diff()
diff_y = c_watabe_eye_data['Gaze point Y'].interpolate(method='linear').diff()
distance_gaze2d = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離

#固視点
diff_x = c_watabe_eye_data['Fixation point X'].interpolate(method='linear').diff()
diff_y = c_watabe_eye_data['Fixation point Y'].interpolate(method='linear').diff()
distance_fixa = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離

#視線3D
diff_x = c_watabe_eye_data['Gaze point 3D X'].interpolate(method='linear').diff()
diff_y = c_watabe_eye_data['Gaze point 3D Y'].interpolate(method='linear').diff()
diff_z = c_watabe_eye_data['Gaze point 3D Z'].interpolate(method='linear').diff()
distance_gaze3d = np.sqrt(diff_x**2 + diff_y**2 + diff_z**2)  # ユークリッド距離

#瞳孔径変化
diff_r = c_watabe_eye_data['Pupil diameter right'].interpolate(method='linear').diff()
diff_l = c_watabe_eye_data['Pupil diameter left'].interpolate(method='linear').diff()
diff_pupil = (diff_r + diff_l) / 2

#視線方向
rx = c_watabe_eye_data['Gaze direction right X'].interpolate(method='linear').diff()
ry = c_watabe_eye_data['Gaze direction right Y'].interpolate(method='linear').diff()
rz = c_watabe_eye_data['Gaze direction right Z'].interpolate(method='linear').diff()
distance_r = np.sqrt(rx**2 + ry**2 + rz**2)
lx = c_watabe_eye_data['Gaze direction left X'].interpolate(method='linear').diff()
ly = c_watabe_eye_data['Gaze direction left Y'].interpolate(method='linear').diff()
lz = c_watabe_eye_data['Gaze direction left Z'].interpolate(method='linear').diff()
distance_l = np.sqrt(lx**2 + ly**2 + lz**2)
distance_dire = (distance_l + distance_r) / 2

#瞳孔位置
rx = c_watabe_eye_data['Pupil position right X'].interpolate(method='linear').diff()
ry = c_watabe_eye_data['Pupil position right Y'].interpolate(method='linear').diff()
rz = c_watabe_eye_data['Pupil position right Z'].interpolate(method='linear').diff()
distance_r = np.sqrt(rx**2 + ry**2 + rz**2)
lx = c_watabe_eye_data['Pupil position left X'].interpolate(method='linear').diff()
ly = c_watabe_eye_data['Pupil position left Y'].interpolate(method='linear').diff()
lz = c_watabe_eye_data['Pupil position left Z'].interpolate(method='linear').diff()
distance_l = np.sqrt(lx**2 + ly**2 + lz**2)
distance_pupilposi = (distance_r + distance_l) / 2


std_individual_c.append(('c_watabe', distance_gaze2d.std(), distance_fixa.std(), distance_gaze3d.std(), diff_pupil.std(), distance_dire.std(), distance_pupilposi.std()))
mean_individual_c.append(('c_watabe', distance_gaze2d.mean(), distance_fixa.mean(), distance_gaze3d.mean(), diff_pupil.mean(), distance_dire.mean(), distance_pupilposi.mean()))

# 結果を出力
std_individual_c = pd.DataFrame(std_individual_c, columns=['Subjects', 'Gaze2D', 'Fixation', 'Gaze3D', 'Pupil', 'GazeDire', 'PupilPosi'])
# 結果を出力
mean_individual_c = pd.DataFrame(mean_individual_c, columns=['Subjects', 'Gaze2D', 'Fixation', 'Gaze3D', 'Pupil', 'GazeDire', 'PupilPosi'])

In [ ]:
std_individual_d = []
mean_individual_d = []
#視線2D
diff_x = d_nakazawa_eye_data['Gaze point X'].interpolate(method='linear').diff()
diff_y = d_nakazawa_eye_data['Gaze point Y'].interpolate(method='linear').diff()
distance_gaze2d = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離

#固視点
diff_x = d_nakazawa_eye_data['Fixation point X'].interpolate(method='linear').diff()
diff_y = d_nakazawa_eye_data['Fixation point Y'].interpolate(method='linear').diff()
distance_fixa = np.sqrt(diff_x**2 + diff_y**2)  # ユークリッド距離

#視線3D
diff_x = d_nakazawa_eye_data['Gaze point 3D X'].interpolate(method='linear').diff()
diff_y = d_nakazawa_eye_data['Gaze point 3D Y'].interpolate(method='linear').diff()
diff_z = d_nakazawa_eye_data['Gaze point 3D Z'].interpolate(method='linear').diff()
distance_gaze3d = np.sqrt(diff_x**2 + diff_y**2 + diff_z**2)  # ユークリッド距離

#瞳孔径変化
diff_r = d_nakazawa_eye_data['Pupil diameter right'].interpolate(method='linear').diff()
diff_l = d_nakazawa_eye_data['Pupil diameter left'].interpolate(method='linear').diff()
diff_pupil = (diff_r + diff_l) / 2

#視線方向
rx = d_nakazawa_eye_data['Gaze direction right X'].interpolate(method='linear').diff()
ry = d_nakazawa_eye_data['Gaze direction right Y'].interpolate(method='linear').diff()
rz = d_nakazawa_eye_data['Gaze direction right Z'].interpolate(method='linear').diff()
distance_r = np.sqrt(rx**2 + ry**2 + rz**2)
lx = d_nakazawa_eye_data['Gaze direction left X'].interpolate(method='linear').diff()
ly = d_nakazawa_eye_data['Gaze direction left Y'].interpolate(method='linear').diff()
lz = d_nakazawa_eye_data['Gaze direction left Z'].interpolate(method='linear').diff()
distance_l = np.sqrt(lx**2 + ly**2 + lz**2)
distance_dire = (distance_l + distance_r) / 2

#瞳孔位置
rx = d_nakazawa_eye_data['Pupil position right X'].interpolate(method='linear').diff()
ry = d_nakazawa_eye_data['Pupil position right Y'].interpolate(method='linear').diff()
rz = d_nakazawa_eye_data['Pupil position right Z'].interpolate(method='linear').diff()
distance_r = np.sqrt(rx**2 + ry**2 + rz**2)
lx = d_nakazawa_eye_data['Pupil position left X'].interpolate(method='linear').diff()
ly = d_nakazawa_eye_data['Pupil position left Y'].interpolate(method='linear').diff()
lz = d_nakazawa_eye_data['Pupil position left Z'].interpolate(method='linear').diff()
distance_l = np.sqrt(lx**2 + ly**2 + lz**2)
distance_pupilposi = (distance_r + distance_l) / 2

std_individual_d.append(('d_nakazawa', distance_gaze2d.std(), distance_fixa.std(), distance_gaze3d.std(), diff_pupil.std(), distance_dire.std(), distance_pupilposi.std()))
mean_individual_d.append(('d_nakazawa', distance_gaze2d.mean(), distance_fixa.mean(), distance_gaze3d.mean(), diff_pupil.mean(), distance_dire.mean(), distance_pupilposi.mean()))

# 結果を出力
std_individual_d = pd.DataFrame(std_individual_d, columns=['Subjects', 'Gaze2D', 'Fixation', 'Gaze3D', 'Pupil', 'GazeDire', 'PupilPosi'])
# 結果を出力
mean_individual_d = pd.DataFrame(mean_individual_d, columns=['Subjects', 'Gaze2D', 'Fixation', 'Gaze3D', 'Pupil', 'GazeDire', 'PupilPosi'])

In [ ]:
a_yuuma_filtered_extracted_eye_data, a_yuuma_filtered_filtered_results = mr.filter_by_std_gaze(a_yuuma_extracted_eye_data, a_yuuma_filtered_results, 36)

In [ ]:
a_yuuma_filtered_filtered_results

In [ ]:
len(a_yuuma_filtered_filtered_results)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# a_yuuma_filtered_results に格納されたタイムスタンプの組に基づいてプロット
fig, axes = plt.subplots(len(a_yuuma_filtered_results), 1, figsize=(14, 7 * len(a_yuuma_filtered_results)))

for i, (start_time, end_time) in enumerate(a_yuuma_filtered_results):
    # -5秒から+5秒のデータを取得
    data = a_yuuma_eye_data[
        (a_yuuma_eye_data['Timestamp'] >= (start_time - pd.Timedelta(seconds=5))) &
        (a_yuuma_eye_data['Timestamp'] <= (end_time + pd.Timedelta(seconds=5)))
    ]

    # 視線移動距離の計算
    diff_x = data['Gaze point X'].interpolate(method='linear').diff()
    diff_y = data['Gaze point Y'].interpolate(method='linear').diff()
    distance = np.sqrt(diff_x**2 + diff_y**2)

    # ハイライトされた区間のデータを抽出
    highlighted_data = data[(data['Timestamp'] >= start_time) & (data['Timestamp'] <= end_time)]
    highlighted_diff_x = highlighted_data['Gaze point X'].interpolate(method='linear').diff()
    highlighted_diff_y = highlighted_data['Gaze point Y'].interpolate(method='linear').diff()
    highlighted_distance = np.sqrt(highlighted_diff_x**2 + highlighted_diff_y**2)

    # プロットの作成
    ax = axes[i]
    color = 'tab:blue'
    ax.set_xlabel('Timestamp')
    ax.set_ylabel('Gaze Movement Distance (pixels)', color=color)
    ax.plot(data['Timestamp'], distance, label='Gaze Movement Distance', color=color)
    ax.tick_params(axis='y', labelcolor=color)

    # ハイライトされた区間のデータをプロット
    ax.plot(highlighted_data['Timestamp'], highlighted_distance, label='Highlighted Gaze Movement Distance', color='tab:orange')
    ax.axvspan(start_time, end_time, color='yellow', alpha=0.3, label='Gesture Performed Segment')

    # Y軸の範囲を設定
    ax.set_ylim(0, 400)  # 上限と下限を適切な値に設定

    # タイトルと凡例
    ax.set_title(f'Gaze Movement Distance and Highlighted Segment {i+1}')
    ax.legend(loc='upper right')

    # グリッドとレイアウト
    ax.grid(True)
    ax.tick_params(axis='x', rotation=45)

fig.tight_layout()

# プロットの保存（PNG形式）
plt.savefig("/Users/hinase/Downloads/a_yuuma_gaze_movement_distance.png", format='png', bbox_inches='tight')

# プロットの表示
plt.show()

In [ ]:
len(c_watabe_filtered_filtered_results)

In [ ]:
# Combine all segments from multiple lists into one list
combined_segments = []
for segments in filtered_overlap:
    combined_segments.extend(segments)

# Sort combined segments
combined_segments.sort()

In [ ]:
combined_segments

In [ ]:
#  pickleファイルを読み込む
with open('/Users/hinase/Downloads/Th-s/a_yuuma_acc_circle_segments2.75.pkl', 'rb') as f:
    data = pickle.load(f)

# 変数を復元する
a_yuuma_tri_segx = data['a_yuuma_tri_segx']
a_yuuma_tri_segy = data['a_yuuma_tri_segy']
a_yuuma_tri_segz = data['a_yuuma_tri_segz']
b_sakamoto_tri_segx = data['b_sakamoto_tri_segx']
b_sakamoto_tri_segy = data['b_sakamoto_tri_segy']
b_sakamoto_tri_segz = data['b_sakamoto_tri_segz']
c_watabe_tri_segx = data['c_watabe_tri_segx']
c_watabe_tri_segy = data['c_watabe_tri_segy']
c_watabe_tri_segz = data['c_watabe_tri_segz']
d_nakazawa_tri_segx = data['d_nakazawa_tri_segx']
d_nakazawa_tri_segy = data['d_nakazawa_tri_segy']
d_nakazawa_tri_segz = data['d_nakazawa_tri_segz']
a_yuuma_tri_overlap = mr.overlap_from_three_segments(a_yuuma_tri_segx, a_yuuma_tri_segy, a_yuuma_tri_segz)
b_sakamoto_tri_overlap = mr.overlap_from_three_segments(b_sakamoto_tri_segx, b_sakamoto_tri_segy, b_sakamoto_tri_segz)
c_watabe_tri_overlap = mr.overlap_from_three_segments(c_watabe_tri_segx, c_watabe_tri_segy, c_watabe_tri_segz)
d_nakazawa_tri_overlap = mr.overlap_from_three_segments(d_nakazawa_tri_segx, d_nakazawa_tri_segy, d_nakazawa_tri_segz)
# 経過時間のフィルタリングを適用
a_yuuma_filtered_overlap = mr.filter_overlap_by_elapsed_time(a_yuuma_tri_overlap, a_yuuma_motion_data, min_time, max_time)
b_sakamoto_filtered_overlap = mr.filter_overlap_by_elapsed_time(b_sakamoto_tri_overlap, b_sakamoto_motion_data, min_time, max_time)
c_watabe_filtered_overlap = mr.filter_overlap_by_elapsed_time(c_watabe_tri_overlap, c_watabe_motion_data, min_time, max_time)
d_nakazawa_filtered_overlap = mr.filter_overlap_by_elapsed_time(d_nakazawa_tri_overlap, d_nakazawa_motion_data, min_time, max_time)
# オーバーラップを統合
a_yuuma_combine_overlap = mr.combine_all_overlaps(a_yuuma_filtered_overlap)
b_sakamoto_combine_overlap = mr.combine_all_overlaps(b_sakamoto_filtered_overlap)
c_watabe_combine_overlap = mr.combine_all_overlaps(c_watabe_filtered_overlap)
d_nakazawa_combine_overlap = mr.combine_all_overlaps(d_nakazawa_filtered_overlap)
# 統合された組からタイムスタンプの組を抽出する
a_yuuma_results =  mr.extract_timestamp_from_overlap(a_yuuma_motion_data, a_yuuma_combine_overlap)
b_sakamoto_results =  mr.extract_timestamp_from_overlap(b_sakamoto_motion_data, b_sakamoto_combine_overlap)
c_watabe_results =  mr.extract_timestamp_from_overlap(c_watabe_motion_data, c_watabe_combine_overlap)
d_nakazawa_results =  mr.extract_timestamp_from_overlap(d_nakazawa_motion_data, d_nakazawa_combine_overlap)
# 各組の時間の長さが2秒以下と5秒以上のものを弾く
a_yuuma_filtered_results = mr.filter_segments_by_time_length(a_yuuma_results, min_time, max_time)
b_sakamoto_filtered_results = mr.filter_segments_by_time_length(b_sakamoto_results, min_time, max_time)
c_watabe_filtered_results = mr.filter_segments_by_time_length(c_watabe_results, min_time, max_time)
d_nakazawa_filtered_results = mr.filter_segments_by_time_length(d_nakazawa_results, min_time, max_time)

In [ ]:
# 区間リストの定義
intervals = [
    (d_nakazawa_eye_data['Timestamp'].min(), circle_start_time, 'Stretch'),
    (circle_start_time, circle_end_time, 'Circle'),
    (circle_end_time, cross_start_time, 'Walking'),
    (cross_start_time, cross_end_time, 'Cross'),
    (cross_end_time, tri_start_time, 'Eating'),
    (tri_start_time, tri_end_time, 'Triangle'),
    (tri_end_time, check_start_time, 'Game'),
    (check_start_time, check_end_time, 'Check'),
    (check_end_time, d_nakazawa_eye_data['Timestamp'].max(), 'Desk work')
]
# 結果をカウントする辞書
interval_counts = {label: 0 for _, _, label in intervals}
a = 0
# 各フィルタされた結果の区間をカウント
for start, end in d_nakazawa_filtered_results:
    for interval_start, interval_end, label in intervals:
        if (interval_start <= start <= interval_end) or (interval_start <= end <= interval_end):
            interval_counts[label] += 1
            a+=1
            break

# 結果を表示
print(interval_counts)
print(a)

In [ ]:
# 結果をカウントする辞書
interval_counts = {label: 0 for _, _, label in intervals}

a = 0
# 各フィルタされた結果の区間をカウント
for start, end in d_nakazawa_filtered_results:
    for interval_start, interval_end, label in intervals:
        if (interval_start <= start <= interval_end) or (interval_start <= end <= interval_end):
            interval_counts[label] += 1
            a+=1
            break

# 結果を表示
print(interval_counts)
print(a)